<a href="https://colab.research.google.com/github/FrnNardes/Top-Insta-Influencers---Linear-Regression/blob/main/LinearRegressionRESTIC36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando bibliotecas e adicionando arquivo csv


In [ ]:
import pandas as pd
from google.colab import files

# Fazer upload do arquivo
print("Por favor, faça o upload do arquivo CSV.")
uploaded = files.upload()

# Exibir os nomes dos arquivos carregados
for filename in uploaded.keys():
    print(f"Arquivo carregado: {filename}")

# Coloca na variável file_path o primeiro arquivo carregado e lê o CSV
file_path = list(uploaded.keys())[0]
df = pd.read_csv(file_path)


Por favor, faça o upload do arquivo CSV.


**Analise Exploratória de Dados (EDA)**

In [ ]:
df.head()

In [ ]:
df.info()

**Convertendo os prefixos 'k', 'm', 'b', '%' para suas respectivas representações numéricas**

In [ ]:
def convert_to_numeric(value):
    if isinstance(value, str):
        if 'k' in value:
            return float(value.replace('k', '')) * 1e3
        elif 'm' in value:
            return float(value.replace('m', '')) * 1e6
        elif 'b' in value:
            return float(value.replace('b', '')) * 1e9
        elif '%' in value:
            return float(value.replace('%', '')) / 100
    return value

df['new_post_avg_like'] = df['new_post_avg_like'].replace('0', np.nan)

columns_to_convert = ['posts', 'followers', 'avg_likes', '60_day_eng_rate', 'new_post_avg_like', 'total_likes']
for col in columns_to_convert:
    df[col] = df[col].apply(convert_to_numeric)

# Verificar o resultado das conversões
df[columns_to_convert].head()

**Algumas estatisticas gerais de cada coluna**

In [ ]:
print(df.describe())
print(df.info())

*Havia um valor = '0' fazendo com que 'new_post_avg_like' ficasse com o tipo object, então transformei o valor em NaN, para ser removido posteriormente no codigo*

In [ ]:
df['new_post_avg_like'].unique()

**Matriz de correlação**

In [ ]:
import seaborn as sns

sns.pairplot(df, kind='scatter', plot_kws={'alpha' : 0.4})


**Removendo alguns outliers**

---


*Fiz diversos testes, e o que obtive a maior precisao foi fazendo IQR apenas na coluna '60_day_eng_rate', 'influence_score' e na variavel dependende de followers, pois a alguns valores muito descrepantes com a tendencia*

In [ ]:
# Colunas utilizadas no processo
numerical_features = ['60_day_eng_rate', 'followers']

# Iteração
for feature in numerical_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Limites inferior e superior
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filtra o DataFrame, removendo os outliers da coluna atual
    df_filtered = df_filtered[(df_filtered[feature] >= lower_bound) & (df_filtered[feature] <= upper_bound)]

# Reseta o índice do DF
df_filtered = df_filtered.reset_index(drop=True)

**Matriz de Correlação após IQR**

In [ ]:
sns.pairplot(df_filtered, kind='scatter', plot_kws={'alpha' : 0.4})

**Rodando o modelo de regressão linear**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

# Selecionar features e variável alvo
features = ['posts', 'avg_likes', '60_day_eng_rate','new_post_avg_like','total_likes', 'influence_score']
target = 'followers'

# Remover valores NaN
df_clean = df_filtered[features + [target]].dropna()

# Dividir em conjuntos de treino e teste
X = df_clean[features]
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e aplicar o StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustar o modelo de regressão linear com dados escalados
model = LinearRegression(fit_intercept=True, copy_X=True, positive=False)
model.fit(X_train_scaled, y_train)

# Prever no conjunto de teste escalado
y_pred = model.predict(X_test_scaled)

# Avaliar o modelo
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Calcular o RMSE usando validação cruzada
scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

# Imprimir os resultados
print("R2:", r2)
print("RMSE:", rmse)
print("MAE:", mae)
print('-----------------------')
print(f"RMSE médio: {rmse_scores.mean():.2f}")
print(f"Desvio padrão do RMSE: {rmse_scores.std():.2f}")
print('-----------------------')
print(f"Obtemos um R² de {r2*100}%, o que significa que o modelo explica ~70% da variação nos dados. E nosso RMSE = {rmse} mostra que o modelo erra em média ~7 milhões de unidades e um MAE de ~5 milhoes. Considerando o alto valor de seguidores dos influenciadores, este valor fica aceitavel.")

**Plotando as relações**


---


*Como temos mais de 3 dimensoes, não podemos visualiza-las em conjunto, mas podemos plotar as relações de cada uma com a variavel dependente e sua reta de regressão linear*

In [ ]:
import matplotlib.pyplot as plt

for feature in features:
    sns.regplot(x=feature, y=target, data=df_clean, scatter_kws={'alpha': 0.4})
    plt.title(f'{feature} vs {target}')
    plt.show()